In [1]:
import import_hack
import core.steps as steps
import core.data_store
import core
from core.data_store.sqlite_table_datastore import SQLiteTableDataStore
from core.data_store.sqlite_table_one_to_many_datastore import SQLiteTableOneToManyDataStore
from core.data_store.file_system_directory_datastore import FileSystemDirectoryDataStore
from core.data_store.numpy_datastore import NumpyDataStore
from core.data_store.stream_ndarray_adapter_datastore import StreamNdarrayAdapterDataStore
from core.quantization.pq_quantizer import PQQuantizer, restore_from_clusters
from core.transformer.bytes_to_ndarray import BytesToNdarray
from core.transformer.ndarray_to_opencvmatrix import NdarrayToOpencvMatrix
from core.transformer.opencvmatrix_to_histogram import OpencvMatrixToHistogram
from core.transformer.opencvmatrix_to_glcm import OpencvMatrixToGLCM
from core.transformer.opencvmatrix_to_lbphistogram import OpencvMatrixToLBPHistogram
from core.search.exhaustive_searcher import ExhaustiveSearcher
from core.search.inverted_multi_index_searcher import InvertedMultiIndexSearcher
from core.common.ds_utils import print_ds_items_info
from core.evaluation.retrieval_perfomance import PrecisionRecallAveragePrecisionEvaluator
from core.evaluation.ground_truth import BrodatzGroundTruth
from core import transformer as trs
from core.transformer.mean_calculator import MeanCalculator
import numpy as np
%load_ext autoreload
%autoreload 2

# Compute descriptors(siftsets)

In [2]:
img_dir_path = r'C:\data\images\brodatz\data.brodatz\size_213x213'
images_ds = FileSystemDirectoryDataStore(dir_path=img_dir_path)
print("images count in '{0}': ".format(img_dir_path), images_ds.get_count())
base_transformers=[BytesToNdarray(), NdarrayToOpencvMatrix()]

images count in 'C:\data\images\brodatz\data.brodatz\size_213x213':  999


In [3]:
glcm_transformers=base_transformers+[OpencvMatrixToGLCM(True)]
glcm_ds= SQLiteTableDataStore("ds_data\imgid_glcm","imgid_glcm")
# steps.transform_step(images_ds, glcm_transformers, glcm_ds)
print_ds_items_info(glcm_ds)

count of items in ds:  999
shape of item[0]:  (65536,)
shape of item[1]:  (65536,)
shape of item[2]:  (65536,)


# Approximate search with inverted multi-index

### Quantize global descriptors

In [4]:
glcmclusters_ds=SQLiteTableDataStore('ds_data\imgid_glcmclusters', 'clusterid_cluster')
quantizer = PQQuantizer(n_clusters=256, n_quantizers=2)
# steps.quantize_step(glcm_ds, quantizer, glcmclusters_ds)
print_ds_items_info(glcmclusters_ds)

count of items in ds:  2
shape of item[0]:  (256, 32768)
shape of item[1]:  (256, 32768)


In [12]:
with glcmclusters_ds:
    items=glcmclusters_ds.get_items_sorted_by_ids()
    for item in items:
        print(item.flags)

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  UPDATEIFCOPY : False
  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  UPDATEIFCOPY : False


### Build inverted multi-index

In [6]:
glcmclusters_ndarray_ds=StreamNdarrayAdapterDataStore(glcmclusters_ds, detect_final_shape_by_first_elem=True)
glcmcluster_centers=glcmclusters_ndarray_ds.get_items_sorted_by_ids()
glcm_ndarray_ds = StreamNdarrayAdapterDataStore(glcm_ds, detect_final_shape_by_first_elem=True)
glcm_X = glcm_ndarray_ds.get_items_sorted_by_ids()
glcm_X_ids=glcm_ndarray_ds.get_ids_sorted()
glcm_imi_searcher=InvertedMultiIndexSearcher(glcm_X, glcm_X_ids, glcmcluster_centers)

### Search approximate neighbors for all bovws(productsiftbincount_ds)

In [7]:
n_nearest=1

In [ ]:
glcm_approximateneighborsids_ds=NumpyDataStore()
steps.search_step(glcm_ds, glcm_imi_searcher, n_nearest, glcm_approximateneighborsids_ds)
print_ds_items_info(glcm_approximateneighborsids_ds, print_shape=False, first_items_to_print=5)

In [22]:
# %%timeit -n3 -r3
lbphistogram_imi_neighborsids_ds=NumpyDataStore()
steps.search_step(lbphistogram_ds, lbphistogram_imi_searcher, n_nearest, lbphistogram_imi_neighborsids_ds)
# print_ds_items_info(lbphistogram_imi_neighborsids_ds, print_shape=False, first_items_to_print=5)

In [26]:
histogram_imi_retrieval_perfomance_ds=NumpyDataStore()
steps.evaluation_step(histogram_imi_neighborsids_ds, perfomance_evaluator, histogram_imi_retrieval_perfomance_ds)

In [27]:
lbphistogram_imi_retrieval_perfomance_ds=NumpyDataStore()
steps.evaluation_step(lbphistogram_imi_neighborsids_ds, perfomance_evaluator, lbphistogram_imi_retrieval_perfomance_ds)

In [31]:
histogram_imi_average_retrieval_perfomance_ds=NumpyDataStore()
steps.averaging_step(histogram_imi_retrieval_perfomance_ds, histogram_imi_average_retrieval_perfomance_ds)

In [32]:
lbphistogram_imi_average_retrieval_perfomance_ds=NumpyDataStore()
steps.averaging_step(lbphistogram_imi_retrieval_perfomance_ds, lbphistogram_imi_average_retrieval_perfomance_ds)

In [166]:
plot_results([histogram_average_retrieval_perfomance_ds, 
              lbphistogram_average_retrieval_perfomance_ds, histogram_imi_average_retrieval_perfomance_ds,
             lbphistogram_imi_average_retrieval_perfomance_ds], labels, 
             1, 'precision', loc='lower left')

[  1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.  14.  15.
  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.]
[ 1.          0.97797798  0.95395395  0.92992993  0.90610611  0.87771104
  0.85113685  0.81969469  0.77922367  0.71741742  0.66511967  0.62137137
  0.58235158  0.54733305  0.51531532  0.48736236  0.46311017  0.44088533
  0.41999895  0.4007007   0.38324038  0.36727637  0.35291814  0.33921421
  0.32656657]
[ 1.          0.93643644  0.88688689  0.8455956   0.80660661  0.77360694
  0.73902474  0.70720721  0.67534201  0.62612613  0.58312858  0.54562896
  0.51274351  0.48305448  0.45699032  0.43374625  0.41253018  0.39328217
  0.37532269  0.35955956  0.34491634  0.33092183  0.31844888  0.30697364
  0.29641642]
[ 0.97097097  0.94294294  0.91324658  0.88288288  0.85065065  0.81715048
  0.78149578  0.74537037  0.71249027  0.65515516  0.60578761  0.56456456
  0.52837453  0.49714     0.469336    0.44413163  0.42271683  0.40279168
  0.38480586  0.36806807  0.35244769  0.3

In [164]:
plot_results([histogram_average_retrieval_perfomance_ds, 
              lbphistogram_average_retrieval_perfomance_ds, histogram_imi_average_retrieval_perfomance_ds,
             lbphistogram_imi_average_retrieval_perfomance_ds], labels, 
             2, 
             'recall',
             loc='lower right')

[  1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.  14.  15.
  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.]
[ 0.1         0.1955956   0.28618619  0.37197197  0.45305305  0.52662663
  0.5957958   0.65575576  0.7013013   0.71741742  0.73163163  0.74564565
  0.75705706  0.76626627  0.77297297  0.77977978  0.78728729  0.79359359
  0.797998    0.8014014   0.8048048   0.80800801  0.81171171  0.81411411
  0.81641642]
[ 0.1         0.18728729  0.26606607  0.33823824  0.4033033   0.46416416
  0.51731732  0.56576577  0.60780781  0.62612613  0.64144144  0.65475475
  0.66656657  0.67627628  0.68548549  0.69399399  0.7013013   0.70790791
  0.71311311  0.71911912  0.72432432  0.72802803  0.73243243  0.73673674
  0.74104104]
[ 0.0970971   0.18858859  0.27397397  0.35315315  0.42532533  0.49029029
  0.54704705  0.5962963   0.64124124  0.65515516  0.66636637  0.67747748
  0.68688689  0.695996    0.704004    0.71061061  0.71861862  0.72502503
  0.73113113  0.73613614  0.74014014  0.7

In [165]:
plot_results([histogram_average_retrieval_perfomance_ds, 
              lbphistogram_average_retrieval_perfomance_ds, histogram_imi_average_retrieval_perfomance_ds,
             lbphistogram_imi_average_retrieval_perfomance_ds], labels,
             3, 'mean average precision',
             loc='lower right')

[  1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.  14.  15.
  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.]
[ 0.1         0.1955956   0.28575242  0.37031198  0.44955122  0.52087254
  0.58682421  0.64308046  0.68525597  0.6968876   0.70644261  0.71507625
  0.72165975  0.72646456  0.72972115  0.73303696  0.73662879  0.73943715
  0.74119681  0.7424931   0.74371337  0.74483267  0.74605564  0.74685226
  0.74755297]
[ 0.1         0.18728729  0.26459793  0.3344678   0.39624958  0.45293961
  0.50137375  0.54492981  0.580343    0.59308574  0.60261345  0.61007091
  0.61593832  0.62047857  0.62476286  0.6287043   0.63166019  0.63432953
  0.6363368   0.63834381  0.64005981  0.64122916  0.64261751  0.64388544
  0.64499455]
[ 0.0970971   0.18778779  0.27140474  0.34845679  0.41812646  0.47962129
  0.53281734  0.57781234  0.61768554  0.62692478  0.63366789  0.63974897
  0.64455377  0.64920128  0.65300508  0.65584542  0.65909573  0.66153706
  0.66387098  0.66573784  0.66715354  0.6

In [54]:
labels=["histogram(bins:256)", "LBP histogram(r:1, points:8, uniform)",
        "histogram(inverted multi-index)", "LBP histogram(inverted multi-index)"]
steps.plotting_step2([histogram_average_retrieval_perfomance_ds, lbphistogram_average_retrieval_perfomance_ds,
                     histogram_imi_average_retrieval_perfomance_ds, lbphistogram_imi_average_retrieval_perfomance_ds], 
                     0, 3, labels, 
                     'n_nearest', 'mAP', save_to_file='hist_lbphist_imihist_imilbphist_25_precision.png')